# Alternative Infill Strategies for Expensive Multi-Objective Optimisation
[A. Rahat](http://emps.exeter.ac.uk/computer-science/staff/aamr201), [R. Everson](http://emps.exeter.ac.uk/computer-science/staff/reverson), and [J. Fieldsend](http://emps.exeter.ac.uk/computer-science/staff/jefields).   
[Department of Computer Science, University of Exeter, UK](http://emps.exeter.ac.uk/computer-science/).

>This repository contains Python code for the infill streategies presented in __Alternative Infill Strategies for Expensive Multi-Objective Optimisation__ by A. Rahat, R. Everson and J. Fieldsend, to appear in GECCO 2017 proceedings. Please refer to the _LICENSE_ before using the code. 

>Preprint repository: https://ore.exeter.ac.uk/repository/handle/10871/27157

>IPython notebook: http://nbviewer.jupyter.org/urls/bitbucket.org/arahat/gecco-2017/raw/6bafce8df19f40f1f0c92584b98efa063a59d594/ReadME.ipynb


## Pre-requisits.

The code here is a __Python3__ implementation of the infill strategies. The following modules are necessary to run the code here. 

* [DEAP](https://github.com/DEAP/deap)
* [Numpy](http://www.numpy.org/)
* [SciPy](https://www.scipy.org/)
* [matplotlib](https://matplotlib.org/2.0.0/index.html)
* [PyDOE](https://pythonhosted.org/pyDOE/)
* [evoalgos](https://ls11-www.cs.tu-dortmund.de/people/swessing/evoalgos/doc/)
* [GPy](https://github.com/SheffieldML/GPy)
* [CMA](https://www.lri.fr/~hansen/html-pythoncma/frames.html)

To install any of these modules, just issue the following command in your terminal. 

`$ pip install module_name`

To install a module from github, use appropriate command. For instance:

`$ pip install git+https://github.com/DEAP/deap`

In addition to these, we used a custom module written in _C_ for dominance comparison. The code is given in the repository. To install the module, use the following command wihtin the _FrontCalc_ directory.

`$ python setup.py install`

> __Note.__ As Python installations differ quite significantly, there may be other dependencies, but these should be standard modules from PyPi. If running the code results in complaints that something could not be imported, then please install the relevant module using _pip_.

## Setting up.

The multi-objective evolutionary optimiser method (_IscaOpt.Optimiser.EMO_) requires a _settings_ dictionary along with the multi-objective function and associated arguments or keyword arguments. We list the the most pertinent settings below. 

* n_dim (int): the number of dimensions in the parameter space.
* n_obj (int): the number of objectives, i.e. objective space dimensions.
* lb (list or numpy array): an array of lower bounds in parameter space (1 in each dimension).
* ub (list or numpy array): an array of upper bounds in parameter space (1 in each dimension).
* ref_vector (list or numpy array): reference vector in the objective space.
* method_name (str): the method to use for performing multi-objective optimisation (deafulats to 'HypI'). Options are:
    - 'HypI'
    - 'MSD'
    - 'DomRank'
    - 'MPoI'
    - 'SMSEGO'
    - 'ParEGO'
* budget (int): the budget on the number of function evaluations.
* n_samples (int): the number of initial samples.
* kern_name (str): the kernel function to be used with Gaussian Processes. Defaults to __'Matern52'__. Please refer to GPy documentation for other options. 
* s_vector (int): the number of scalarisation vectors for ParEGO.
* maxfevals (int): the maximum number of function evaluations for infill criterion optimisation using CMA-ES. Defaults to $20000d$, where $d$ is the number of dimensions in parameter space.  
* multisurrogate (bool): whether to use a multi-surrogate approach or not. Defaults to __False__, i.e. use a mono-surrogate approach. 
* cma_options (dict): dictionary of settings for Hansen's CMA-ES code. See [CMA-ES documentation](https://www.lri.fr/~hansen/html-pythoncma/frames.html) for more details on avaialble options.
* cma_sigma (float): the extent of the standard deviation for CMA-ES. See [CMA-ES documentation](https://www.lri.fr/~hansen/html-pythoncma/frames.html) for details. 
* init_file (str): intial design file. It should be a __.npz__ file with 'arr_0' set to decision variables matrix $X \in \mathbb{R}^{M \times n}$ and 'arr_1' for corresponding function response vector $\mathbf{f} \in \mathbb{R}^{M \times D}$ (please refer to the paper for details on notations).
* visualise (bool): it allows basic visualisations. Only available for the following cases: __n_obj=2__; __n_obj=1 and n_dim=2__; __n_obj=1 and n_dim=1__.

> __Notes__<br />
> * This package can be used for single objective Bayesian optimisation. To do so, specify the method by setting **method_name** to **'EGO'**, and of course **n_obj** to **1** with an appropriate function. <br />
> * For one-dimensional search space and a single objective problem, we just use a grid-search instead of CMA-ES. 




## Running the optimiser.

To run the optimiser it is sufficient to define an objective function that may produce a response given a decision vector, and use appropriate settings to call the multi-objective evolutionary optimiser method (_IscaOpt.Optimiser.EMO_). Here we give an example of using the optimiser. 

### An example.

_Problem description_: Use the optimiser to solve $2$-objective __DTLZ2__ problem, starting with $65$ initial LHS samples and a budget of $100$. 

In [ ]:
from IPython.display import clear_output
# example set up
import numpy as np
# import optimiser codes
import IscaOpt

settings = {\
    'n_dim': 6,\
    'n_obj': 2,\
    'lb': np.zeros(6),\
    'ub': np.ones(6),\
    'ref_vector': [2.5]*2,\
    'method_name': 'DomRank',\
    'budget':67,\
    'n_samples':65,\
    'visualise':True,\
    'multisurrogate':False}

# function settings
from deap import benchmarks as BM
fun = BM.dtlz2
args = (2,) # number of objectives as argument

# optimise
res = IscaOpt.Optimiser.EMO(fun, args, settings=settings)
clear_output()

Simulation settings. 
{'n_dim': 6, 'n_obj': 2, 'lb': array([ 0.,  0.,  0.,  0.,  0.,  0.]), 'ub': array([ 1.,  1.,  1.,  1.,  1.,  1.]), 'ref_vector': [2.5, 2.5], 'method_name': 'DomRank', 'budget': 67, 'n_samples': 65, 'visualise': True, 'multisurrogate': False}
<class 'IscaOpt.mono_surrogate.DomRank'>   Mat52.       |  value  |  constraints  |  priors
  variance     |    1.0  |      +ve      |        
  lengthscale  |   (6,)  |      +ve      |        
method used:  DomRank
Episode:  1
[ 0.  0.  0.  0.  0.  0.] [ 1.  1.  1.  1.  1.  1.]
new X
Total time:  0.00036249160766601565
Creating surrogate...
Optimiser name:  lbfgs
Optimization restart 1/10, f = 80.83841343982473
Optimization restart 2/10, f = 191.9899941529032
Optimization restart 3/10, f = 658.4031291399291
Optimization restart 4/10, f = 658.4031250261241
Optimization restart 5/10, f = 92.23100465830481
Optimization restart 6/10, f = 658.4031291617879
Optimization restart 7/10, f = 658.4031291617881
Optimization restart 8/10,

 /home/alma/miniconda3/lib/python3.6/site-packages/cma/constraints_handler.py:108: FutureWarning:comparison to `None` will result in an elementwise object comparison in the future.


   89    801 -1.785369878880601e-02 1.3e+02 2.39e-04  5e-06  4e-04 0:00.8
termination on tolfun=1e-07 (Wed Aug 23 11:47:27 2017)
final/bestever f-value = -1.785370e-02 -1.785370e-02
incumbent solution: [0.025501145312356279, 0.98154574298415687, 0.58179886850296292, 0.4104260386983547, 0.22202702557355911, 0.35041512319940565]
std deviation: [2.6280961151394418e-05, 0.00044281930896927763, 4.671281369377532e-06, 3.3427334760061777e-05, 0.00027170750589558681, 5.5209328698942779e-05]
(9_w,18)-aCMA-ES (mu_w=5.4,w_1=30%) in dimension 6 (seed=597790, Wed Aug 23 11:47:27 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1    820 -3.004365824623242e-02 1.0e+00 2.46e-01  2e-01  3e-01 0:00.0
    2    838 -1.110896398173781e-02 1.3e+00 2.37e-01  2e-01  3e-01 0:00.0
    3    856 -1.945403172719042e-02 1.7e+00 2.91e-01  2e-01  4e-01 0:00.1
   71   2080 -3.493219549982316e-02 8.5e+02 1.68e-03  6e-06  5e-03 0:01.3
termination on tolfun=1e-07 after 1 restart (Wed Aug 

   95  17458 -2.729735899508012e-02 1.1e+02 3.86e-04  7e-06  8e-04 0:00.9
termination on tolfun=1e-07 after 9 restarts (Wed Aug 23 11:47:47 2017)
final/bestever f-value = -2.729736e-02 -3.493222e-02
incumbent solution: [0.20419207199999778, 0.0059157976339943288, 0.38193944906528449, 0.66592061822900361, 0.99576082396179255, 0.88877787102083039]
std deviation: [2.6604814071221042e-05, 0.00079382104160407815, 6.8320089764355481e-06, 3.883287023388373e-05, 0.00050617992009931016, 6.3426086747241212e-05]
(24_w,48)-aCMA-ES (mu_w=13.4,w_1=14%) in dimension 6 (seed=597799, Wed Aug 23 11:47:47 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1  17507 -3.531863940425045e-03 1.0e+00 4.94e-02  5e-02  5e-02 0:00.0
    2  17555 -4.711043154013497e-03 1.5e+00 5.54e-02  4e-02  6e-02 0:00.1
    3  17603 -3.833527388008657e-03 1.7e+00 6.76e-02  5e-02  8e-02 0:00.1
   71  20867 -1.983293647733960e-02 6.1e+03 7.02e-02  2e-05  1e-01 0:03.2
termination on tolfun=1e-07 afte

    3  41477 -3.420868177457052e-03 1.4e+00 6.06e-03  5e-03  7e-03 0:00.0
   91  42269 -1.705304282975832e-02 4.5e+01 9.25e-05  2e-06  8e-05 0:00.9
termination on tolfun=1e-07 after 18 restarts (Wed Aug 23 11:48:16 2017)
final/bestever f-value = -1.705304e-02 -3.493222e-02
incumbent solution: [0.99999999983620014, 0.88776202829120643, 0.15398921289327863, 0.51979092893973644, 0.19274365399802931, 1.6681429115344365e-10]
std deviation: [1.0289295985246196e-05, 8.3720102183165525e-05, 1.9170240705916528e-06, 1.3996911332272006e-05, 4.7471961488774029e-05, 2.5024288690478288e-05]
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=597808, Wed Aug 23 11:48:16 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1  42279 -1.618700119067852e-03 1.0e+00 2.27e-02  2e-02  2e-02 0:00.0
    2  42288 -3.275827129743391e-03 1.3e+00 2.54e-02  2e-02  3e-02 0:00.0
    3  42297 -6.358403684458299e-03 1.6e+00 3.17e-02  3e-02  4e-02 0:00.0
  100  43170 -1.705304481321051e

    1 105637 -5.495967011588885e-03 1.0e+00 8.36e-03  8e-03  9e-03 0:00.0
    2 105647 -7.797236803972664e-03 1.4e+00 1.12e-02  1e-02  1e-02 0:00.0
    3 105657 -1.177831556873109e-02 1.7e+00 1.66e-02  2e-02  2e-02 0:00.0
  100 106627 -1.983293480918965e-02 1.3e+02 3.64e-04  4e-06  5e-04 0:01.0
  101 106637 -1.983293490510859e-02 1.4e+02 3.23e-04  3e-06  4e-04 0:01.0
termination on tolfun=1e-07 after 27 restarts (Wed Aug 23 11:49:33 2017)
final/bestever f-value = -1.983293e-02 -3.493222e-02
incumbent solution: [0.7659388665207455, 0.78658864053145106, 0.49389935341330499, 0.16198173501018309, 0.67935165717744495, 0.98646351406781796]
std deviation: [1.7892295877760729e-05, 0.00040009905478607043, 3.1827366802261998e-06, 2.6526602560936053e-05, 0.00033484059434774868, 9.9429911621169489e-05]
(23_w,46)-aCMA-ES (mu_w=12.9,w_1=15%) in dimension 6 (seed=597817, Wed Aug 23 11:49:33 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1 106684 -5.916261936510148e-

    1   6527 -1.463099729692677e-03 1.0e+00 4.61e-03  4e-03  6e-03 0:00.0
    2   6555 -1.526614600094611e-03 1.5e+00 6.35e-03  6e-03  8e-03 0:00.1
    3   6583 -1.816075401725410e-03 1.5e+00 9.29e-03  8e-03  1e-02 0:00.1
   64   8291 -2.587022293134050e-02 2.0e+02 8.62e-03  5e-05  1e-02 0:01.6
termination on maxiter=64.0 after 5 restarts (Wed Aug 23 11:49:56 2017)
final/bestever f-value = -2.587023e-02 -2.587023e-02
incumbent solution: [0.20108110724574091, 0.53485292528962103, 0.38550594245212355, 0.64001004932558403, 0.16560081874163429, 0.89591292818787127]
std deviation: [0.00022687879914860877, 0.0051991425200086606, 5.2149179155713958e-05, 0.00024239314621070127, 0.011226504282927255, 0.00053213669128298843]
(36_w,72)-aCMA-ES (mu_w=19.7,w_1=10%) in dimension 6 (seed=625936, Wed Aug 23 11:49:56 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1   8364 -1.405697356999033e-02 1.0e+00 2.80e-01  2e-01  3e-01 0:00.1
    2   8436 -1.291198462731398e-02 

    1  27397 -2.198271463061983e-03 1.0e+00 7.60e-03  7e-03  8e-03 0:00.0
    2  27408 -2.436845083678245e-03 1.3e+00 8.71e-03  8e-03  9e-03 0:00.0
    3  27419 -2.610496459209322e-03 1.5e+00 1.17e-02  1e-02  1e-02 0:00.0
   77  28233 -5.254020747581981e-03 7.5e+01 1.32e-03  5e-05  4e-03 0:00.9
termination on tolfun=1e-07 after 14 restarts (Wed Aug 23 11:50:17 2017)
final/bestever f-value = -5.254021e-03 -2.587023e-02
incumbent solution: [4.9479375394471935e-10, 0.031412659442082595, 3.2157315411009516e-09, 0.99999999796490335, 0.40221743187415032, 0.99999997850379718]
std deviation: [5.0338481088978061e-05, 0.0035284633397435054, 5.6468448635353933e-05, 0.0001667362367829532, 0.001836018565167906, 0.00033729084157698]
(19_w,38)-aCMA-ES (mu_w=10.8,w_1=17%) in dimension 6 (seed=625945, Wed Aug 23 11:50:18 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1  28272 -1.900300937331968e-08 1.0e+00 7.04e-03  7e-03  8e-03 0:00.0
termination on tolfun=1e-07 afte

    2  48514 -1.543063288759406e-04 1.4e+00 1.44e-02  1e-02  2e-02 0:00.1
    3  48547 -4.283697634685270e-04 1.4e+00 2.17e-02  2e-02  2e-02 0:00.1
   76  50956 -1.285513371539871e-02 1.7e+03 1.21e-02  1e-05  2e-02 0:02.3
termination on tolfun=1e-07 after 23 restarts (Wed Aug 23 11:50:46 2017)
final/bestever f-value = -1.285513e-02 -2.587023e-02
incumbent solution: [0.86530449369083096, 0.99562018696750865, 0.99999999996801692, 0.6207175512471409, 0.34076520784163356, 0.99999997983506628]
std deviation: [3.8786318811454637e-05, 0.024685601980770751, 1.4727440959064254e-05, 5.1964181461832254e-05, 0.016761630666509719, 0.00013135189620283125]
(11_w,23)-aCMA-ES (mu_w=6.7,w_1=25%) in dimension 6 (seed=625954, Wed Aug 23 11:50:46 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1  50980 -8.134365877585209e-03 1.0e+00 2.22e-02  2e-02  2e-02 0:00.0
    2  51003 -8.283717145321525e-03 1.4e+00 2.77e-02  3e-02  3e-02 0:00.0
    3  51026 -8.474128160799791e-03 1.

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1  96105 -1.900256194863212e-02 1.0e+00 2.37e-01  2e-01  3e-01 0:00.1
    2  96249 -2.128478373860901e-02 1.7e+00 3.09e-01  2e-01  4e-01 0:00.3
    3  96393 -2.003698125720667e-02 2.2e+00 3.09e-01  2e-01  4e-01 0:00.4
   25  99561 -2.585126840268182e-02 8.7e+01 4.29e-01  3e-03  3e-01 0:03.4


In the Figure above, the blue crosses show the initial samples, and the solid circle show the newly sampled solutions with darker colours showing earlier samples. The black encircled solid is the latest sample.

## Errata

* Equation (5) should be as follows. 
    $\alpha(\mathbf{x}, f^*) = \int_{-\infty}^{\infty}I(\mathbf{x}, f^*)P(\hat{f}|\mathbf{x},\mathcal{D})d\hat{f} = \sigma(\mathbf{x})(s\Phi(s) + \phi(s))$.

## Contact

For any comments, queries or suggestions, please send an email to: __a.a.m.rahat@exeter.ac.uk__